In [ ]:
import requests
import config
import ast
from datetime import datetime, timedelta
import psycopg

class Attempt:
    def __init__(self, user_id, oauth_consumer_key, lis_result_sourcedid, lis_outcome_service_url, is_correct, attempt_type, created_at):
        self.user_id = user_id
        self.user_id = user_id
        self.is_correct = is_correct
        self.attempt_type = attempt_type
        self.created_at = created_at
        self.oauth_consumer_key = oauth_consumer_key
        self.lis_result_sourcedid = lis_result_sourcedid
        self.lis_outcome_service_url = lis_outcome_service_url


In [13]:
def get_response():
    end_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    start_date = end_date - timedelta(days=1)

    api_url = config.api_url
    q_params = {
        'client': config.client,
        'client_key': config.client_key,
        'start': start_date,
        'end': end_date
    }
    response = requests.get(api_url, params=q_params)
    return response.json()

def parse_passback_params(passback_params):
    required_keys = ['oauth_consumer_key', 'lis_result_sourcedid', 'lis_outcome_service_url']
    result = ast.literal_eval(passback_params)
    if all(key in result for key in required_keys):
        return result
    return None

def validate_attempt(attempt):
    required_keys = ['lti_user_id', 'is_correct', 'attempt_type', 'created_at']
    if not all(key in attempt for key in required_keys):
        return False
    if not isinstance(attempt.get('lti_user_id'), str):
        return False
    
    attempt_type = attempt.get('attempt_type')
    is_correct = attempt.get('is_correct')
    if attempt_type == 'run':
        if is_correct is not None:
            return False
    elif attempt_type == 'submit':
        if not isinstance(is_correct, bool):
            return False
    else:
        return False
    return True
    
def create_attempts_list(attempt, passback_params):
    user_id = attempt.get('lti_user_id')
    oauth_consumer_key = passback_params.get('oauth_consumer_key')
    lis_result_sourcedid = passback_params.get('lis_result_sourcedid')
    lis_outcome_service_url = passback_params.get('lis_outcome_service_url')
    is_correct = attempt.get('is_correct')
    attempt_type = attempt.get('attempt_type')
    created_at = attempt.get('created_at')
    attempt_o = Attempt(user_id, oauth_consumer_key, lis_result_sourcedid, lis_outcome_service_url, is_correct, attempt_type, created_at)
    return attempt_o
    

def main():
    attempts = []
    attempts_data = get_response()
    for attempt in attempts_data:
        if not validate_attempt(attempt):
            continue
        passback_params = parse_passback_params(attempt.get('passback_params'))
        if not passback_params:
            continue
        attempts.append(create_attempts_list(attempt, passback_params))
    for a in attempts:
        print(a.__dict__)


    


main()

{'user_id': 'cd2edf026711eab6997c5706d1c6cc8d', 'is_correct': None, 'attempt_type': 'run', 'created_at': '2026-01-05 05:09:55.244622', 'oauth_consumer_key': '', 'lis_result_sourcedid': 'course-v1:SkillFactory+DSPR-2.0+14JULY2021:lms.skillfactory.ru-a17f589a8511443287f7826c7e744965:cd2edf026711eab6997c5706d1c6cc8d', 'lis_outcome_service_url': 'https://lms.skillfactory.ru/courses/course-v1:SkillFactory+DSPR-2.0+14JULY2021/xblock/block-v1:SkillFactory+DSPR-2.0+14JULY2021+type@lti+block@a17f589a8511443287f7826c7e744965/handler_noauth/grade_handler'}
{'user_id': 'cd2edf026711eab6997c5706d1c6cc8d', 'is_correct': None, 'attempt_type': 'run', 'created_at': '2026-01-05 05:11:07.408759', 'oauth_consumer_key': '', 'lis_result_sourcedid': 'course-v1:SkillFactory+DSPR-2.0+14JULY2021:lms.skillfactory.ru-a17f589a8511443287f7826c7e744965:cd2edf026711eab6997c5706d1c6cc8d', 'lis_outcome_service_url': 'https://lms.skillfactory.ru/courses/course-v1:SkillFactory+DSPR-2.0+14JULY2021/xblock/block-v1:SkillFac